In [39]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn import linear_model as lm
import xgboost as xgb
from scipy.stats import uniform, randint
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
import lightgbm as lgb

from sklearn.model_selection import train_test_split

In [40]:
import pandas as pd

In [27]:
dataset = pd.read_csv('../../dataset/baseball_train_final5_longhit.csv')

In [28]:
dataset.columns

Index(['선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷',
       '사구', '고4', '삼진', '병살', '희타', '희비', '투구', 'barrel', '타율', 'LG', 'KIA',
       'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 'SSG', '홈경기수', '원정경기수', '장타'],
      dtype='object')

In [29]:
dataset = dataset[dataset['타수'] >= 30]
dataset = dataset[(dataset["장타"] != 0)]

In [30]:
X_feature = ['선발', '타수', '득점', '안타', '2타', '3타', '홈런',
            '루타', '타점','볼넷', '삼진', '희타',
            '병살','barrel',
            'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 
            'SSG', '홈경기수', '원정경기수', ]

y_feature = '장타'

In [32]:
X = dataset[X_feature]
y = dataset.loc[:, y_feature]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1)

In [14]:
train_ds = lgb.Dataset(X_train, label = y_train) 
test_ds = lgb.Dataset(X_val, label = y_val)

In [35]:
params = {'learning_rate': 0.01, 
          'max_depth': 16, 
          'boosting': 'gbdt', 
          'objective': 'regression', 
          'metric': 'mse', 
          'is_training_metric': True, 
          'num_leaves': 144, 
          'feature_fraction': 0.9, 
          'bagging_fraction': 0.7, 
          'bagging_freq': 5, 
          'seed':2018}

In [16]:
model = lgb.train(params, train_ds, 1000, test_ds, verbose_eval=100, early_stopping_rounds=100)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021136 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3647
[LightGBM] [Info] Number of data points in the train set: 6230, number of used features: 26
[LightGBM] [Info] Start training from score 0.623026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Ligh

In [17]:
predict_test = model.predict(X_val)

In [18]:
mean_absolute_error(predict_test, y_val)

0.05401418063209394

In [48]:
lgbm_model = LGBMRegressor()

gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['dart'], # for better accuracy -> try dart
    'objective' : ['regression'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    #'colsample_bytree' : [0.64, 0.65, 0.66],
    #'subsample' : [0.7,0.75],
    #'reg_alpha' : [1,1.2],
    #'reg_lambda' : [1,1.2,1.4],
    }

In [50]:
grid_tree = GridSearchCV(lgbm_model, param_grid=gridParams, cv=5, refit=True)
grid_tree.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'boosting_type': ['dart'],
                         'learning_rate': [0.005, 0.01], 'max_bin': [255, 510],
                         'n_estimators': [8, 16, 24],
                         'num_leaves': [6, 8, 12, 16],
                         'objective': ['regression'], 'random_state': [500]})

In [51]:
grid_tree.best_score_

0.04602462988413294

In [52]:
len(dataset)

6923